In [81]:
import pulp
import pandas as pd

counties_df = pd.read_csv('counties_table.csv')
counties_df = counties_df.reset_index()
counties_df = counties_df.drop(counties_df.index[0])
counties_df


,index,county,population,Demographic-_white majority,long,lat,FIPS
1,1,Asotin County,"22,508",87.4,-117.202791,46.191771,53003
2,2,Benton County,"212,791",65.2,-119.511334,46.239760,53005
3,3,Chelan County,"79,926",65.6,-120.618828,47.869219,53007
4,4,Clallam County,"77,805",79.2,-123.927711,48.049117,53009
5,5,Clark County,"516,779",72.7,-122.482311,45.779198,53011
6,6,Columbia County,"4,026",84.9,-117.907825,46.297961,53013
7,7,Cowlitz County,"111,956",79.3,-122.680273,46.193215,53015
8,8,Douglas County,"44,192",58.9,-119.691707,47.735896,53017
9,9,Ferry County,"7,448",70.4,-118.516324,48.470397,53019
10,10,Franklin County,"98,678",38.2,-118.898475,46.535046,53021


In [87]:
print(counties_df.columns)

counties_df['population'] = counties_df['population'].str.replace(',', '', regex=True).astype(int)


Index(['index', 'county', 'population', 'Demographic-_white majority', 'long',
       'lat', 'FIPS'],
      dtype='object')


In [103]:
counties_df['population'] = counties_df['population'].astype("int")
counties_df['lat'] = counties_df['lat'].astype("float")
counties_df['long'] = counties_df['long'].astype("float")

counties_df.info()

max_district = 10
max_population = counties_df['population'].sum() * 0.15
counties = counties_df['county']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 1 to 38
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   index                        38 non-null     int64  
 1   county                       38 non-null     object 
 2   population                   38 non-null     int64  
 3   Demographic-_white majority  38 non-null     float64
 4   long                         38 non-null     float64
 5   lat                          38 non-null     float64
 6   FIPS                         38 non-null     int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 2.2+ KB


In [105]:
import statistics as stats

counties_Lat_Dic = dict(zip(counties_df.county, counties_df.lat))
counties_Long_Dic = dict(zip(counties_df.county, counties_df.long))
counties_Pop_Dic = dict(zip(counties_df.county, counties_df.population))

def compactness(district):
    lat_list = list( map(counties_Lat_Dic.get, district) )
    long_list = list( map(counties_Long_Dic.get, district) )
    
    lat_sd = stats.stdev(lat_list)
    long_sd = stats.stdev(long_list)
    
    compact_score = lat_sd + long_sd
    
    return compact_score

def total_pop(district):
    pop_list = list( map(counties_Pop_Dic.get, district) )
    population = sum(pop_list)
    return population

from itertools import chain, combinations
min_len = 2
max_len = 3
possible_districts = list(chain.from_iterable(combinations(counties, i) for i in range(min_len, max_len+1)))
len(possible_districts)

9139

In [107]:

#create a binary variable to state that a district is used
x = pulp.LpVariable.dicts('district', possible_districts, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

redistrict_model = pulp.LpProblem("Redistricting Model", pulp.LpMinimize)

#specify the maximum number of districts
redistrict_model += sum([x[district] for district in possible_districts]) == max_district, "Maximum_number_of_districts"

#specify the population max
for district in possible_districts:
    redistrict_model += total_pop(district) * x[district] <= max_population, f"Max_people_in_{district}"

#A county can be assigned to one and only one district
for county in counties:
    redistrict_model += sum([x[district] for district in possible_districts if county in district]) == 1, "Must_zone_%s"%county

redistrict_model += sum([compactness(district) * x[district] for district in possible_districts])
print(redistrict_model)

redistrict_model.solve()
count_districts = 0
print("The choosen districts are out of a total of %s:"%len(possible_districts))
for district in possible_districts:
    if x[district].value() == 1.0:
        count_districts += 1
        print(district)


print(count_districts)


/home/daf03834-2cd7-40ad-bc31-d3b51e4b5c9f/.local/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/daf03834-2cd7-40ad-bc31-d3b51e4b5c9f/.local/lib/python3.9/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/3169459150ad4c25909e7a6b561d3680-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/3169459150ad4c25909e7a6b561d3680-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9183 COLUMNS
At line 81593 RHS
At line 90772 BOUNDS
At line 99912 ENDATA
Problem MODEL has 9178 rows, 9139 columns and 44992 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.03 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.11   (Wallclock seconds):       0.13

The choosen districts are out of a total of 9139:
('Benton County', 'Franklin County', 'Yakima County')
('Chelan County', 'Kittitas County', 'Yakima County')
('Clallam County', 'Island County', 'San 